In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import math

In [ ]:
data = pd.read_csv('/content/dataset.csv', sep=',')
pd.options.mode.chained_assignment = None
len_data = len(data)
print(len_data)

1665


In [ ]:
data.info()
print('-------------------------------')
print('Count of records: {}'.format(len_data))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ClientID                    1665 non-null   object
 1   CycleNumber                 1665 non-null   int64 
 2   Group                       1665 non-null   int64 
 3   CycleWithPeakorNot          1665 non-null   int64 
 4   ReproductiveCategory        1665 non-null   int64 
 5   LengthofCycle               1665 non-null   int64 
 6   MeanCycleLength             1665 non-null   object
 7   EstimatedDayofOvulation     1665 non-null   object
 8   LengthofLutealPhase         1665 non-null   object
 9   FirstDayofHigh              1665 non-null   object
 10  TotalNumberofHighDays       1665 non-null   object
 11  TotalHighPostPeak           1665 non-null   object
 12  TotalNumberofPeakDays       1665 non-null   object
 13  TotalDaysofFertility        1665 non-null   obje

Patient Information


In [ ]:
patient_data_row_counter = Counter()
for id, row in data['ClientID'].iteritems():
    patient_data_row_counter[row] += 1
per_user_rows = sorted(set(patient_data_row_counter.values()))
print('Unique patients: {}'.format(data['ClientID'].nunique()))
print('Data row count per patient between {} - {}'.format(per_user_rows[0], per_user_rows[-1]))

Unique patients: 159
Data row count per patient between 1 - 45


<ipython-input-4-21d88a28f2be>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id, row in data['ClientID'].iteritems():


Cycle length info.

In [ ]:
print('Cycle length between {} - {}'.format(data['LengthofCycle'].min(), data['LengthofCycle'].max()))

Cycle length between 18 - 54


Leutal Phase info. The luteal phase is a phase in the menstrual cycle that occurs after ovulation and before the start of menstruation. It is primarily associated with the secretion of progesterone, a hormone that helps prepare the uterine lining (endometrium) for a potential pregnancy.

In [ ]:
clean_luteal_data = data[data['LengthofLutealPhase'] != ' ']
clean_luteal_data['LengthofLutealPhase'] = pd.to_numeric(clean_luteal_data['LengthofLutealPhase'], downcast='unsigned')
print('Luteal phase length between {} - {} days.'.format(clean_luteal_data['LengthofLutealPhase'].min(), clean_luteal_data['LengthofLutealPhase'].max()))
print('{} records are good from {}.'.format(len(clean_luteal_data), len_data))

Luteal phase length between 1 - 41 days.
1514 records are good from 1665.


Ovulation day's info: Ovulation is a key event in the menstrual cycle of a woman and refers to the release of a mature egg (ovum) from one of the ovaries. It typically occurs around the middle of the menstrual cycle.

In [ ]:
clean_ovulation_data = data[data['EstimatedDayofOvulation'] != ' ']
clean_ovulation_data['EstimatedDayofOvulation'] = pd.to_numeric(clean_ovulation_data['EstimatedDayofOvulation'], downcast='unsigned')
print('Estimated day of ovulation between {} - {} days.'.format(clean_ovulation_data['EstimatedDayofOvulation'].min(),clean_ovulation_data['EstimatedDayofOvulation'].max()))
print('{} records are good from {}.'.format(len(clean_ovulation_data), len_data))

Estimated day of ovulation between 6 - 29 days.
1515 records are good from 1665.


Menses info.: "Menses" refers to the menstrual phase of the menstrual cycle, which is the portion of the cycle during which a woman experiences menstrual bleeding or her period.

In [ ]:
clean_menses_data = data[data['LengthofMenses'] != ' ']
clean_menses_data['LengthofMenses'] = pd.to_numeric(clean_menses_data['LengthofMenses'], downcast='unsigned')
print('Length of menses between {} - {} days.'.format(clean_menses_data['LengthofMenses'].min(),
                                                      clean_menses_data['LengthofMenses'].max()))
print('{} records are good from {}.'.format(len(clean_menses_data), len_data))

Length of menses between 2 - 15 days.
1661 records are good from 1665.


Data cleaning

In [ ]:
clean_data = data[data['LengthofLutealPhase'] != ' ']
clean_data = clean_data[clean_data['LengthofMenses'] != ' ']
clean_data = clean_data[clean_data['EstimatedDayofOvulation'] != ' ']

clean_data = clean_data[['ClientID', 'LengthofCycle', 'LengthofLutealPhase', 'LengthofMenses' , 'EstimatedDayofOvulation']]
for i, key in enumerate(patient_data_row_counter.keys()):
    clean_data = clean_data.replace(key, i)
clean_data.info()
print('-------------------------------')
print('Count of records: {}'.format(len(clean_data)))
clean_data.to_csv('clean_dataset.csv', sep=',', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1663
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ClientID                 1512 non-null   int64 
 1   LengthofCycle            1512 non-null   int64 
 2   LengthofLutealPhase      1512 non-null   object
 3   LengthofMenses           1512 non-null   object
 4   EstimatedDayofOvulation  1512 non-null   object
dtypes: int64(2), object(3)
memory usage: 70.9+ KB
-------------------------------
Count of records: 1512


**Importing libraries to carry out ANN**

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import math

**Importing clean dataset**

In [ ]:
dataset = pd.read_csv('clean_dataset.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

**Splitting Dataset into Training and Testing set**

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

**BUILDING THE ANN**


In [ ]:
ann = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(units=1)
])

**Compiling the ANN**

In [ ]:
ann.compile(optimizer='adam', loss='mean_squared_error')

**Training the ANN model on Training set**

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=150, validation_split=0.2)

Epoch 1/150
31/31 [==============================] - 2s 13ms/step - loss: 244.0827 - val_loss: 216.5130
Epoch 2/150
31/31 [==============================] - 0s 5ms/step - loss: 187.0419 - val_loss: 146.0646
Epoch 3/150
31/31 [==============================] - 0s 5ms/step - loss: 108.5860 - val_loss: 64.5577
Epoch 4/150
31/31 [==============================] - 0s 5ms/step - loss: 40.9189 - val_loss: 23.7966
Epoch 5/150
31/31 [==============================] - 0s 5ms/step - loss: 22.2941 - val_loss: 18.5487
Epoch 6/150
31/31 [==============================] - 0s 5ms/step - loss: 19.3139 - val_loss: 16.1323
Epoch 7/150
31/31 [==============================] - 0s 4ms/step - loss: 15.8086 - val_loss: 14.2997
Epoch 8/150
31/31 [==============================] - 0s 5ms/step - loss: 14.2538 - val_loss: 12.8951
Epoch 9/150
31/31 [==============================] - 0s 5ms/step - loss: 12.3505 - val_loss: 11.7218
Epoch 10/150
31/31 [==============================] - 0s 5ms/step - loss: 11.6896 - v

**Predicting the Result**

In [ ]:
y_pred = ann.predict(X_test)

10/10 [==============================] - 0s 2ms/step


In [ ]:
y_pred = ann.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))

10/10 [==============================] - 0s 2ms/step


In [ ]:
print("R-squared Score:", r2)

R-squared Score: 0.9427939991975066


In [ ]:
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.8619748853346083
